# Clash of Clans CWL
## Data pipeline build

* Your Name: Nolan Clark
* Name of API: Clash of Clans API
* Link to main page of API documentation: https://developer.clashofclans.com/#/documentation

In [1]:
# Imports go here
import requests
import pandas as pd
import numpy as np
import MyKeys

import json
import datetime

import CWL_Functions as cf
from collections import Counter
from collections import defaultdict

import time

# API Token used with the documented Authorization header. 
# The headers are the necessary params of each endpoint
MyToken = MyKeys.MyToken
headers = {'authorization': 'Bearer '+(MyToken), 'Accept': 'application/json'}
base = 'https://api.clashofclans.com/v1/'

In [2]:
start_all= time.time()
LeagueBins = {'Bronze League III': [],
 'Bronze League II': [],
 'Bronze League I': [],
 'Silver League III': [],
 'Silver League II': [],
 'Silver League I': [],
 'Gold League III': [],
 'Gold League II': [],
 'Gold League I': [],
 'Crystal League III': [],
 'Crystal League II': [],
 'Crystal League I': [],
 'Master League III': [],
 'Master League II': [],
 'Master League I': [],
 'Champion League III': [],
 'Champion League II': [],
 'Champion League I': []}

In [3]:
tag_bin = {'Bronze League III': [],
 'Bronze League II': [],
 'Bronze League I': [],
 'Silver League III': [],
 'Silver League II': [],
 'Silver League I': [],
 'Gold League III': [],
 'Gold League II': [],
 'Gold League I': [],
 'Crystal League III': [],
 'Crystal League II': [],
 'Crystal League I': [],
 'Master League III': [],
 'Master League II': [],
 'Master League I': [],
 'Champion League III': [],
 'Champion League II': [],
 'Champion League I': []}

In [4]:
def search_clans(limit, clanPoints='',minMem='', after=''):
    
    if clanPoints !='':
        clanPoints = f'minClanPoints={clanPoints}'
        
    if minMem !='':
        minMem = f'minMembers={minMem}'
    
    if after != '':
        after= f'&after={after}'
    
    url=f'https://api.clashofclans.com/v1/clans?{minMem}{clanPoints}&limit={limit}{after}'
    response = requests.get(url, headers=headers, timeout=30)
    print(response.status_code, response.reason)
    
    top_clans=[i['tag'].strip('#') for i in response.json()['items']]
    
    #top_clans=response.json()
    
    try:
        after = response.json()['paging'].get('cursors')['after']
    except:
        after = False
    
    return top_clans, after
    

In [5]:
def meltLeague(ClanTag):
    url='https://api.clashofclans.com/v1/clans/%23'+ClanTag
    response = requests.get(url, headers=headers, timeout=30)
    #print(response.status_code, response.reason)
    league= response.json()['warLeague']['name']
    
    
    return league

In [6]:
def grabTagsBig(ClanTag):
    collect = []
    
    url='https://api.clashofclans.com/v1/clans/%23'+ClanTag+'/currentwar/leaguegroup'
    response = requests.get(url, headers=headers, timeout=30)
    #print(response.status_code, response.reason)
    
    
    # range of 7 so that only CWL with all rounds will be pulled
    for i in range(7):
        collect.extend(response.json().get('rounds')[i].get('warTags'))
    
    collDict = collect
    
    return collDict

In [7]:
start = time.time()
bucket = []
clanPointInts = [20000,30000,40000,50000]

for points in clanPointInts:
    
    after_page = ''
    while after_page != False:
        all_tags = search_clans(limit=100,clanPoints=points,minMem='',after=after_page)
        bucket.extend(all_tags[0])
        after_page = all_tags[1]
        
end = time.time()
print(end - start)

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
6.153577089309692


In [8]:
len(bucket)

3328

In [9]:
top_clans = pd.unique(bucket)

In [10]:
start = time.time()
inWar_bin = []
for i in top_clans:
    j = cf.status15(i)
    if j != False:
        inWar_bin.append(i)
end = time.time()
print(end - start)

814.8809509277344


In [11]:
for clan in inWar_bin:
    tag_bin[meltLeague(clan)].append(clan)

In [13]:
# LARGE GRAB

leagueNames=list(tag_bin.keys())

for league in leagueNames:
    print(league)
    for tag in tag_bin[league]:
        try:
            rounds = grabTagsBig(tag)
            LeagueBins[league].extend(rounds)
        except (TypeError, IndexError):
            pass

Bronze League III
Bronze League II
Bronze League I
Silver League III
Silver League II
Silver League I
Gold League III
Gold League II
Gold League I
Crystal League III
Crystal League II
Crystal League I
Master League III
Master League II
Master League I
Champion League III
Champion League II
Champion League I


# Diagnostics 

In [14]:
end_all = time.time()
print(end_all - start_all)

1550.5044167041779


In [15]:
{key: (len(LeagueBins[key])) for key in  LeagueBins.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 0,
 'Silver League II': 56,
 'Silver League I': 168,
 'Gold League III': 1064,
 'Gold League II': 1904,
 'Gold League I': 3864,
 'Crystal League III': 6272,
 'Crystal League II': 9184,
 'Crystal League I': 17192,
 'Master League III': 12488,
 'Master League II': 7616,
 'Master League I': 4928,
 'Champion League III': 5096,
 'Champion League II': 2996,
 'Champion League I': 728}

In [16]:
{key: (len(pd.unique(LeagueBins[key]))) for key in  LeagueBins.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 0,
 'Silver League II': 28,
 'Silver League I': 84,
 'Gold League III': 504,
 'Gold League II': 952,
 'Gold League I': 1932,
 'Crystal League III': 3080,
 'Crystal League II': 4592,
 'Crystal League I': 8484,
 'Master League III': 6076,
 'Master League II': 3780,
 'Master League I': 2408,
 'Champion League III': 2352,
 'Champion League II': 1400,
 'Champion League I': 420}

In [17]:
{key: (len(tag_bin[key])) for key in  tag_bin.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 0,
 'Silver League II': 1,
 'Silver League I': 3,
 'Gold League III': 19,
 'Gold League II': 34,
 'Gold League I': 69,
 'Crystal League III': 112,
 'Crystal League II': 164,
 'Crystal League I': 308,
 'Master League III': 223,
 'Master League II': 136,
 'Master League I': 88,
 'Champion League III': 92,
 'Champion League II': 55,
 'Champion League I': 17}

In [18]:
{key: (len(pd.unique(tag_bin[key]))) for key in  tag_bin.keys()}

{'Bronze League III': 0,
 'Bronze League II': 0,
 'Bronze League I': 0,
 'Silver League III': 0,
 'Silver League II': 1,
 'Silver League I': 3,
 'Gold League III': 19,
 'Gold League II': 34,
 'Gold League I': 69,
 'Crystal League III': 112,
 'Crystal League II': 164,
 'Crystal League I': 308,
 'Master League III': 223,
 'Master League II': 136,
 'Master League I': 88,
 'Champion League III': 92,
 'Champion League II': 55,
 'Champion League I': 17}

In [19]:
cwlRoundsBins={key: list(pd.unique(LeagueBins[key])) for key in  LeagueBins.keys()}

In [20]:
# write the updated file
with open("PipelineRunOne.json","w") as write_file:
    json.dump(cwlRoundsBins, write_file)